In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
zri = pd.read_csv('../data/zri_multifamily_v2.csv')

In [5]:
zri['zip'] = zri['zip'].astype('string')
zri['zip'] = ["0"+z if len(z)<5 else z for z in zri['zip']]
zri['year'] = pd.to_datetime(zri['year-month'],format="%Y-%m").dt.year
zri['year'] = zri['year'].astype('string')
zri['year-month'] = pd.to_datetime(zri['year-month'],format="%Y-%m")

In [6]:
zri[zri['State']=='NJ'].head()

,zip,City,State,Metro,CountyName,year-month,zri,year
13536,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-01-01,1415.0,2014
13537,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-02-01,1415.0,2014
13538,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-03-01,1433.0,2014
13539,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-04-01,1434.0,2014
13540,07002,Bayonne,NJ,New York-Newark-Jersey City,Hudson County,2014-05-01,1436.0,2014


In [7]:
zri.shape

(93744, 8)

In [8]:
#zri['CountyName'][~zri['CountyName'].map(lambda s: s.endswith(' County'))].value_counts()

In [9]:
bds = pd.read_csv('../../../zillow_orientation/bds2018_cty.csv')

In [10]:
bds.head(2)

,year,st,cty,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,1,1,396,402,5343,5049,95,24.643,62,16.083,1437,555,882,10.992,28.461,870,742,128,14.696,17.231,567,11.230,34.462,40,40,104
1,1978,1,3,1435,1534,14527,13962,312,21.239,182,12.389,3354,1228,2126,8.795,24.022,2319,1143,1176,8.187,16.609,1035,7.413,33.219,144,145,539


In [11]:
bds.shape

(133250, 27)

In [12]:
bds[['year','st','cty']].duplicated().sum()

0

In [13]:
bds[['year','cty']].duplicated().sum()

119720

In [14]:
bds = bds[['year','st','cty','firms','job_creation_rate','job_destruction_rate']]

In [15]:
bds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133250 entries, 0 to 133249
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   year                  133250 non-null  int64 
 1   st                    133250 non-null  int64 
 2   cty                   133250 non-null  int64 
 3   firms                 133250 non-null  object
 4   job_creation_rate     133250 non-null  object
 5   job_destruction_rate  133250 non-null  object
dtypes: int64(3), object(3)
memory usage: 6.1+ MB


In [16]:
#bds[(bds['year']<=2019) & (bds['year']>=2014)]
bds.columns = ['year','st','cty','total_firms','job_creation_rate','job_destruction_rate']
bds

,year,st,cty,total_firms,job_creation_rate,job_destruction_rate
0,1978,1,1,396,28.461,17.231
1,1978,1,3,1435,24.022,16.609
2,1978,1,5,442,24.467,11.637
3,1978,1,7,187,20.133,19.367
4,1978,1,9,413,20.209,12.750
...,...,...,...,...,...,...
133245,2018,56,45,177,11.053,12.426
133246,2018,56,998,(D),(D),(D)
133247,2018,56,999,134,12.920,12.775
133248,2018,97,998,0,.,.


In [18]:
bds_startup = pd.read_csv('../../../zillow_orientation/bds2018_cty_fage.csv')

In [19]:
#(bds_startup['year']<=2019) & (bds_startup['year']>=2014) 
bds_startup = bds_startup[(bds_startup['fage']=='a) 0')]

In [20]:
bds_startup

,year,st,cty,fage,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,estabs_exit_rate,job_creation,job_creation_births,job_creation_continuers,job_creation_rate_births,job_creation_rate,job_destruction,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,1,1,a) 0,81,81,421,211,81,200.000,0,0.000,421,421,0,200.000,200.000,0,0,0,0.000,0.000,421,200.000,0.000,0,0,0
5,1978,1,3,a) 0,259,261,991,496,261,200.000,0,0.000,991,991,0,200.000,200.000,0,0,0,0.000,0.000,991,200.000,0.000,0,0,0
10,1978,1,5,a) 0,51,51,214,107,51,200.000,0,0.000,214,214,0,200.000,200.000,0,0,0,0.000,0.000,214,200.000,0.000,0,0,0
15,1978,1,7,a) 0,27,27,159,80,27,200.000,0,0.000,159,159,0,200.000,200.000,0,0,0,0.000,0.000,159,200.000,0.000,0,0,0
20,1978,1,9,a) 0,56,56,225,113,56,200.000,0,0.000,225,225,0,200.000,200.000,0,0,0,0.000,0.000,225,200.000,0.000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666225,2018,56,45,a) 0,10,10,28,14,10,200.000,0,0.000,28,28,0,200.000,200.000,0,0,0,0.000,0.000,28,200.000,0.000,0,0,0
666230,2018,56,998,a) 0,0,0,0,0,0,NaN,0,NaN,0,0,0,.,.,0,0,0,.,.,0,.,.,0,0,0
666235,2018,56,999,a) 0,0,0,0,0,0,NaN,0,NaN,0,0,0,.,.,0,0,0,.,.,0,.,.,0,0,0
666240,2018,97,998,a) 0,0,0,0,0,0,NaN,0,NaN,0,0,0,.,.,0,0,0,.,.,0,.,.,0,0,0


In [21]:
bds_startup = bds_startup[['year','st','cty','firms']]

In [22]:
bds_startup.columns = ['year','st','cty','startup_firms']

In [23]:
bds_merged = pd.merge(bds,bds_startup,how='inner')

In [24]:
bds_merged = bds_merged[~bds_merged['total_firms'].map(lambda s: s.startswith('('))]

In [25]:
bds_merged.shape

(132527, 7)

In [26]:
bds_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132527 entries, 0 to 133249
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   year                  132527 non-null  int64 
 1   st                    132527 non-null  int64 
 2   cty                   132527 non-null  int64 
 3   total_firms           132527 non-null  object
 4   job_creation_rate     132527 non-null  object
 5   job_destruction_rate  132527 non-null  object
 6   startup_firms         132527 non-null  object
dtypes: int64(3), object(4)
memory usage: 8.1+ MB


In [27]:
bds_merged['year_usable'] = bds_merged['year']+2

In [28]:
bds_merged['year_usable'].unique()

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [29]:
bds_merged = bds_merged[(bds_merged['year_usable']<=2019) & (bds_merged['year_usable']>=2014)]

In [30]:
bds_merged

,year,st,cty,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable
110500,2012,1,1,731,10.885,11.053,35,2014
110501,2012,1,3,3718,14.967,12.992,280,2014
110502,2012,1,5,413,7.714,14.761,9,2014
110503,2012,1,7,250,12.393,9.457,12,2014
110504,2012,1,9,601,13.101,10.389,26,2014
...,...,...,...,...,...,...,...,...
129994,2017,56,43,307,12.816,16.938,11,2019
129995,2017,56,45,187,12.464,14.893,7,2019
129997,2017,56,999,128,21.946,18.451,0,2019
129998,2017,97,998,0,0.000,200.000,0,2019


In [32]:
cty_map = pd.read_csv('../../../zillow_orientation/label_county.csv')

In [33]:
#cty_map = cty_map.drop(['geoid','availability'],axis=1)

In [34]:
cty_map

,state,county,geoid,name,availability
0,1,0,1000,Alabama,2
1,2,0,2000,Alaska,2
2,4,0,4000,Arizona,2
3,5,0,5000,Arkansas,2
4,6,0,6000,California,2
...,...,...,...,...,...
3275,56,39,56039,"Teton, WY",1
3276,56,41,56041,"Uinta, WY",1
3277,56,43,56043,"Washakie, WY",1
3278,56,45,56045,"Weston, WY",1


In [35]:
bds_merged = pd.merge(bds_merged,cty_map,how='left',left_on=['st','cty'],right_on=['state','county'])

In [36]:
bds_merged = bds_merged.dropna()
bds_merged = bds_merged.drop(['state','county','st','cty','availability'],axis=1)

In [37]:
bds_merged['_state'] = bds_merged['name'].map(lambda s: s[-2:])

In [38]:
bds_merged['_county_label'] = bds_merged['name'].map(lambda s: s.split(',')[0])

In [39]:
bds_merged

,year,total_firms,job_creation_rate,job_destruction_rate,startup_firms,year_usable,geoid,name,_state,_county_label
0,2012,731,10.885,11.053,35,2014,1001.0,"Autauga, AL",AL,Autauga
1,2012,3718,14.967,12.992,280,2014,1003.0,"Baldwin, AL",AL,Baldwin
2,2012,413,7.714,14.761,9,2014,1005.0,"Barbour, AL",AL,Barbour
3,2012,250,12.393,9.457,12,2014,1007.0,"Bibb, AL",AL,Bibb
4,2012,601,13.101,10.389,26,2014,1009.0,"Blount, AL",AL,Blount
...,...,...,...,...,...,...,...,...,...,...
19355,2017,1627,13.632,11.981,143,2019,56039.0,"Teton, WY",WY,Teton
19356,2017,460,8.674,10.371,23,2019,56041.0,"Uinta, WY",WY,Uinta
19357,2017,307,12.816,16.938,11,2019,56043.0,"Washakie, WY",WY,Washakie
19358,2017,187,12.464,14.893,7,2019,56045.0,"Weston, WY",WY,Weston


In [40]:
bds_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19176 entries, 0 to 19359
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  19176 non-null  int64  
 1   total_firms           19176 non-null  object 
 2   job_creation_rate     19176 non-null  object 
 3   job_destruction_rate  19176 non-null  object 
 4   startup_firms         19176 non-null  object 
 5   year_usable           19176 non-null  int64  
 6   geoid                 19176 non-null  float64
 7   name                  19176 non-null  object 
 8   _state                19176 non-null  object 
 9   _county_label         19176 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 1.6+ MB


In [48]:
bds_merged.to_csv('./../data/bds_engineered_features_yearly_county.csv')

In [42]:
#bds_merged[bds_merged['name'].map(lambda s: s.endswith('NJ'))]

In [43]:
#bds_merged['name'].value_counts()

In [44]:
#bds_merged[bds_merged['name'].map(lambda s: s.endswith('VA'))].head(500)

In [45]:
#bds_merged[bds_merged['name'].map(lambda s: s.endswith('AK'))].head(500)

In [46]:
bds_merged[['_state','_county_label']]

,_state,_county_label
0,AL,Autauga
1,AL,Baldwin
2,AL,Barbour
3,AL,Bibb
4,AL,Blount
...,...,...
19355,WY,Teton
19356,WY,Uinta
19357,WY,Washakie
19358,WY,Weston


In [47]:
zri[['State','CountyName']]

,State,CountyName
0,MA,Hampden County
1,MA,Hampden County
2,MA,Hampden County
3,MA,Hampden County
4,MA,Hampden County
...,...,...
93739,AK,Matanuska Susitna Borough
93740,AK,Matanuska Susitna Borough
93741,AK,Matanuska Susitna Borough
93742,AK,Matanuska Susitna Borough
